In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers
!pip install chromadb

In [1]:
from langchain_community.document_loaders import JSONLoader

In [2]:
diff_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

msg_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].msg',
    text_content=False)

msg_data = msg_loader.load()

language_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].language',
    text_content=False)

language_data = language_loader.load()


## Import OpenAI model

In [3]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=300, chunk_overlap=0) for language in languages]

language_dict = {'java': 0, 'cpp': 1, 'csharp': 2, 'python': 3, 'javascript': 4}

In [4]:
from tqdm import tqdm
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    diff_split += splitters[language_dict[language_data[i].page_content]].split_documents([doc])

Processing documents: 100%|██████████| 500000/500000 [01:08<00:00, 7344.55it/s]


In [5]:
len(diff_split)

2774108

In [10]:
diff_split[0]

Document(page_content='diff --git a/butterknife-runtime/src/main/java/butterknife/internal/Utils.java \nppp b/butterknife-runtime/src/main/java/butterknife/internal/Utils.java', metadata={'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\rag_data.json', 'seq_num': 1})

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [9]:
from langchain_community.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_data, embeddings, persist_directory="./chroma_db")

In [12]:
db.persist()

In [13]:
test_diff_loader = JSONLoader(
    file_path='../data/msg_nngen_nmt_codebert_chatgpt.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()

In [15]:
query = test_diff_data[0].page_content
documents = splitters[0].create_documents([query])

In [23]:
def similarity_search(documents):
    # Initialize an empty dictionary to store aggregate scores for each candidate ID
    aggregate_scores = {}
    
    # Iterate through each document in the documents list
    for document in documents:
        # Apply similarity search function to the document
        results = db.similarity_search_with_relevance_scores(document.page_content)
        # Iterate through the results for each document
        for candidate_doc in results:
            id = candidate_doc[0].metadata['seq_num']
            score = candidate_doc[1]
            # Update the aggregate score for the candidate ID
            aggregate_scores[id] = aggregate_scores.get(id, 0) + score
    
    # Find the candidate ID with the highest aggregate score
    max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)
    return max_candidate_id - 1

In [24]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data), total=len(test_diff_data), desc="Processing documents"):
    query = test_data.page_content
    documents = splitters[i//1000].create_documents([query])
    similar_diff.append(diff_data[similarity_search(documents)])

Processing documents:   3%|▎         | 173/5000 [00:09<07:24, 10.87it/s]C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='diff --git a/channels/cw.m3u \nppp b/channels/cw.m3u#EXTM3U-# EXTINF:- 1 tvg-id=""tvg-name=""tvg-logo=" https://i.imgur.com/EMGIQYP.png"group-title="", Dutch Caribbean TV-https://nlengine1.cur.watch/dctv/auto/dctv/720p/chunks.m3u8#EXTINF:- 1 tvg-id=""tvg-name=""tvg-logo=" https://i.imgur.com/jmYvx2s.jpg"group-title="", Nos Païs \nhttp://highvolume04.streampartner.nl:1935/nos_pais_24_7/smil:livestream.stream.smil/chunklist_w1798703532_b2000000.m3u8#EXTINF:- 1 tvg-id=""tvg-name=""tvg-logo=" https://i.imgur.com/9D9G4co.png"group-title="", TeleCuraçao \n', metadata={'seq_num': 466720, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\rag_data.json'}), -0.0066844730595567725), (Document(page_content='diff --git a/channels/my.m3u \npp